In [36]:
%matplotlib inline

import json
import math

import numpy
import matplotlib.pyplot as plot
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.grid_search import GridSearchCV#

import os, fnmatch

from bs4 import BeautifulSoup
import re

In [37]:
mypath = '/data/share/lab05data/'

In [38]:
base = fnmatch.filter(os.listdir(mypath), 'base_*.txt')

In [39]:
test_ids = [514, 3489, 9, 2570, 3084, 3086, 1040, 1113, 3603, 2582, 1559, 2591, 2081, 2602, 1281, 1289, 1076, 569, 2618, 572, 2622, 1599, 2100, 579, 581, 586, 1720, 78, 1616, 3665, 174, 85, 598, 599, 89, 1626, 2139, 3600, 2144, 1121, 1123, 2374, 2066, 3178, 1131, 2067, 624, 3433, 1139, 1658, 1659, 2173, 1151, 1664, 3719, 2696, 2185, 1932, 3214, 656, 1174, 3224, 665, 1691, 160, 161, 3041, 3239, 1704, 683, 1651, 1200, 2225, 2232, 695, 3256, 698, 1211, 1215, 1728, 3268, 2758, 3783, 715, 1230, 1232, 2258, 1806, 219, 1244, 1757, 808, 1262, 2799, 3824, 60, 1779, 1441, 1270, 1697, 257, 3332, 1798, 775, 2313, 1294, 273, 275, 2324, 3349, 3352, 1331, 1831, 3368, 3293, 307, 2864, 2353, 2579, 3892, 2869, 2362, 2876, 1343, 3904, 322, 836, 2885, 3398, 3913, 1355, 3405, 1360, 2386, 3925, 2902, 3415, 857, 347, 144, 2806, 3940, 999, 402, 2921, 1909, 1911, 377, 1406, 1407, 384, 2950, 903, 1928, 3468, 910, 1423, 3472, 2450, 406, 409, 2973, 2462, 1439, 2464, 2977, 2896, 1956, 1961, 1437, 427, 429, 1524, 950, 958, 758, 3233, 3524, 3013, 2504, 3530, 3020, 464, 2517, 3030, 3545, 336, 989, 3550, 479, 1951, 482, 2535, 2538, 1501, 3570, 2036, 887, 1016, 2200]
test = []
for i in test_ids:
    test.append('test_' + str(i) + '.txt')

In [40]:
base_texts = []
for text in base:
    with open(mypath + '/' + text, 'r', encoding='utf-8') as f:
        base_texts.append(f.read())

raw_test_texts = []
for text in test:
    with open(mypath + '/' + text, 'r', encoding='utf-8') as f:
        raw_test_texts.append(f.read())

In [41]:
norm_base_texts = []
for text in base_texts:
    text = BeautifulSoup(text).get_text()
    norm_base_texts.append(text.lower())
    
norm_test_texts = []
for text in raw_test_texts:
    text = BeautifulSoup(text).get_text()
    norm_test_texts.append(text.lower())

/opt/anaconda/envs/bd9/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /opt/anaconda/envs/bd9/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


## Все слова в кучу

In [42]:
import pymystem3
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

mystem = pymystem3.Mystem()
wordnet_lemmatizer = WordNetLemmatizer()

In [43]:
# from nltk.tokenize import RegexpTokenizer
# tokenizer = RegexpTokenizer('[A-Za-zА-Яа-яёЁ]+')

# def tokenize(text):
#     tokens = tokenizer.tokenize(text)
#     for i in tokens:
#         if len(i)>2:
#             yield m.lemmatize(i)[0]

In [44]:
def text_to_wordlist(text):
    text = re.sub('n\'t', ' not', text)
    text = re.sub('[^a-zA-Zа-яА-Я]', ' ', text)
    words = text.lower().split()
    return words

In [45]:
train_texts = []
for item in norm_base_texts:
    new_item = text_to_wordlist(item)
    lemm_mystem = [mystem.lemmatize(x)[0] for x in new_item]
    tokens_stem = [wordnet_lemmatizer.lemmatize(x, pos=wordnet.VERB) for x in lemm_mystem]
    train_texts.append(tokens_stem)

In [46]:
test_texts = []
for item in norm_test_texts:
    new_item = text_to_wordlist(item)
    lemm_mystem = [mystem.lemmatize(x)[0] for x in new_item]
    tokens_stem = [wordnet_lemmatizer.lemmatize(x, pos=wordnet.VERB) for x in lemm_mystem]
    test_texts.append(tokens_stem)

In [47]:
stop_words = ["еще", "него", "сказать", "а", "ж", "нее", "со", "без", "же", "ней", "совсем", "более", "жизнь", 
              "нельзя", "так", "больше", "за", "нет", "такой", "будет", "зачем", "ни", "там", "будто", "здесь", 
              "нибудь", "тебя", "бы", "и", "никогда", "тем", "был", "из", "ним", "теперь", "была", "из", "за",
              "них", "то", "были", "или", "ничего", "тогда", "было", "им", "но", "того", "быть", "иногда", "ну", 
              "тоже", "в", "их", "о", "только", "вам", "к", "об", "том", "вас", "кажется", "один", "тот", "вдруг",
              "как", "он", "три", "ведь", "какая", "она", "тут", "во", "какой", "они", "ты", "вот", "когда", "опять",
              "у", "впрочем", "конечно", "от", "уж", "все", "которого", "перед", "уже", "всегда", "которые", "по",
              "хорошо", "всего", "кто", "под", "хоть", "всех", "куда", "после", "чего", "всю", "ли", "потом", "человек",
              "вы", "лучше", "потому", "чем", "г", "между", "почти", "через", "где", "меня", "при", "что", "говорил",
              "мне", "про", "чтоб", "да", "много", "раз", "чтобы", "даже", "может", "разве", "чуть", "два", "можно",
              "с", "эти", "для", "мой", "сам", "этого", "до", "моя", "свое", "этой", "другой", "мы", "свою", "этом",
              "его", "на", "себе", "этот", "ее", "над", "себя", "эту", "ей", "надо", "сегодня", "я", "ему", "наконец",
              "сейчас", "если", "нас", "сказал", "есть", "не", "сказала"]

In [48]:
for sentence in train_texts:
    for word in sentence:
        if len(word) <= 3:
            sentence.remove(word)
        if word in stop_words and word in sentence:
            sentence.remove(word)
            
for sentence in test_texts:
    for word in sentence:
        if len(word) <= 3:
            sentence.remove(word)
        if word in stop_words and word in sentence:
            sentence.remove(word)

In [49]:
%%time
from gensim.models.word2vec import Word2Vec 

params = {}

params['size'] = 300  # итоговая размерность вектора каждого слова
params['min_count'] = 5  # минимальная частотность слова, чтобы оно попало в модель
params['workers'] = 8     # количество ядер вашего процессора, чтоб запустить обучение в несколько потоков


model = Word2Vec(train_texts, **params)

CPU times: user 20 ms, sys: 4 ms, total: 24 ms
Wall time: 26 ms


In [50]:
model.init_sims(replace=True)

## Схожесть текстов

In [51]:
import numpy as np

def text_to_vec(words, model, size):
    text_vec = np.zeros((size,), dtype="float32")
    n_words = 0

    index2word_set = set(model.wv.vocab.keys())
    for word in words:
        if word in index2word_set:
            n_words = n_words + 1
            text_vec = np.add(text_vec, model[word])
    
    if n_words != 0:
        text_vec /= n_words
    return text_vec

In [52]:
from tqdm import tqdm_notebook as tqdm
def texts_to_vecs(texts, model, size):
    texts_vecs = np.zeros((len(texts), size), dtype="float32")
    
    for i, text in tqdm(enumerate(texts),):
        texts_vecs[i] = text_to_vec(text, model, size)

    return texts_vecs

In [53]:
train_vecs = texts_to_vecs(train_texts, model, params['size'])

/opt/anaconda/envs/bd9/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


In [54]:
test_vecs = texts_to_vecs(test_texts, model, params['size'])

/opt/anaconda/envs/bd9/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


In [55]:
import operator

def cosine_similarity(vec1, vec2):

    def dot_product2(v1, v2):
        return sum(map(operator.mul, v1, v2))

    def vector_cos5(v1, v2):
        prod = dot_product2(v1, v2)
        len1 = math.sqrt(dot_product2(v1, v1))
        len2 = math.sqrt(dot_product2(v2, v2))
        return prod / (len1 * len2)

    return vector_cos5(vec1, vec2)

In [56]:
similarity = []

for test in test_vecs:
    tmp = 0
    for train in train_vecs:
        tmp += cosine_similarity(train, test)
    similarity.append(tmp)

In [57]:
mean = sum(similarity)/len(similarity)

In [58]:
is_sim = [1 if x >= mean else 0 for x in similarity]

In [59]:
ones = [i[0] for i in list(zip(test_ids, is_sim)) if i[1] == 1]
zeros = [i[0] for i in list(zip(test_ids, is_sim)) if i[1] == 0]

In [60]:
fin_json =  { "defined" : ones, "other" : zeros} 

In [61]:
import json
with open('../lab05.json', 'w') as outfile:
    json.dump(fin_json, outfile)